In [23]:
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel,GPT2Model, GPT2Config, DataCollatorForLanguageModeling,BertTokenizer,BertModel,BertConfig
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import evaluate
%pwd

'/home/ubuntu/ssk/LiGOFwlr/tutorial'

In [24]:
dataset = load_dataset("/home/ubuntu/ssk/openwebtext")

/home/ubuntu/miniconda3/envs/pytorch/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [25]:
dataset["eval"] = dataset["train"].select(range(20000,20100))
dataset["train"] = dataset["train"].select(range(10000))

In [26]:
tokenizer: GPT2Tokenizer = GPT2Tokenizer.from_pretrained(
        "/home/ubuntu/ssk/LiGOFwlr/tutorial/GPT2-tokenizer"
    )

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=50)

In [27]:
tokenized_datasets["train"].set_format(type="torch", columns=["input_ids", "attention_mask"])

tokenized_datasets["eval"].set_format(type="torch", columns=["input_ids", "attention_mask"])

In [28]:
tokenized_datasets["train"][0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [29]:
tokenized_datasets["train"][0]['attention_mask']

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])

In [30]:
def shift_labels_right(inputs):
    labels = inputs["input_ids"].clone()
    # labels[-1] = -100 # ignore_index
    # labels[0:-1] = inputs["input_ids"][1:].clone()
    inputs["labels"] = labels
    return inputs
final_dataset = tokenized_datasets.map(shift_labels_right)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [31]:
final_dataset["train"][0]

{'input_ids': tensor([50256, 13924,    12,   559,    12, 35784,    11, 25051,   357, 18474,
             8,  1377, 45591,  4970,    11,  1319, 44556,   287,  2356,   290,
         44787,   379,  1204,    11,  7342,  7519,   290, 20669,  2513,  1497,
           422,   257,  2214,  4436,  3217,  1755,   706,   257, 21402,  3315,
          1074, 23724,   262,  1989,    11,  2282,   340,   373,  5213,   546,
          2324,    13,   198,   198,   464,  2551,  1364,  8100,  5953,  8366,
         34428,   298,  2986, 33708, 42095,   355,   262,   691,  6253,   379,
           262,  4436,   284,   651,   262,  3871,   832,   262,  1755,    13,
           198,   198, 18474,  7317,  2098,    11,  1912,   319, 10275,   351,
           617,   286,   262,  7519,    11,   326,   262,  1578,  7973,  6149,
           262, 21402,  3274, 22225,   290,  7929,  4816,   284, 36316,    13,
          2102,    11, 21402,  5953, 36831,  2269,   861,   402,  2926,    82,
            11,   257,  6253,   508,   

In [32]:
config_gpt2 = GPT2Config(n_positions=128,
                    n_embd=128,
                    n_layer=6,
                    n_head=8)
model= GPT2LMHeadModel(config_gpt2)

In [33]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 128)
    (wpe): Embedding(128, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=128, out_features=50257, bias=False)
)

In [34]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(final_dataset["train"], shuffle=True, batch_size=8,pin_memory=True)
eval_dataloader = DataLoader(final_dataset["eval"], batch_size=8,pin_memory=True)

In [35]:
from torch.optim import AdamW
import torch
optimizer = AdamW(model.parameters(),lr=1e-5)

In [36]:
device = torch.device("cuda:1")

In [88]:
model.to(device)
from tqdm.auto import tqdm

num_epochs =3 
num_training_steps = num_epochs * len(train_dataloader)
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/3750 [00:00<?, ?it/s]

In [83]:
encoded_inputs = tokenizer.encode(["1"])
encoded_inputs_tensor = torch.LongTensor(encoded_inputs).to(device)

In [84]:
generated_results = model.generate(encoded_inputs_tensor.unsqueeze(0))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [85]:
tokenizer.decode(generated_results.squeeze().cpu().numpy())

'<|endoftext|>1 the the the the the the the the the the the the the the the the the the'

In [86]:
import torch
import torchvision

In [87]:
torchvision.datasets.flowers102

<module 'torchvision.datasets.flowers102' from '/home/ubuntu/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/datasets/flowers102.py'>